# Custom Chatbot Project

## Dataset Selection and Rationale

For this custom chatbot project, I have chosen the **2023 Fashion Trends dataset** (`2023_fashion_trends.csv`). This dataset is perfectly suited for creating a fashion advice chatbot that can provide expert recommendations and insights about current fashion trends.

**Why this dataset is appropriate:**

1. **Rich Text Content**: The dataset contains detailed descriptions of fashion trends, styling advice, and expert commentary from fashion industry professionals, providing comprehensive context for fashion-related queries.

2. **Specific Domain Knowledge**: Unlike a general-purpose chatbot, this dataset enables the creation of a specialized fashion advisor that can provide current, trend-specific recommendations based on 2023 fashion industry insights.

3. **Practical Use Case**: Users can ask about specific trends, styling advice, color combinations, and seasonal fashion recommendations - creating clear value-add scenarios where custom data significantly improves response quality.

4. **Authoritative Sources**: The data comes from reputable fashion publications like Refinery29, InStyle, Glamour, and Vogue, ensuring high-quality, expert-level fashion advice.

5. **Sufficient Data Volume**: With detailed trend descriptions from multiple sources, the dataset provides enough content to demonstrate meaningful performance improvements over generic responses.

**Target Scenario**: A fashion advice chatbot that can provide personalized styling recommendations, trend insights, and expert fashion guidance based on current 2023 fashion industry knowledge.

## Data Wrangling

In this section, I load the chosen dataset into a pandas dataframe and create the required `"text"` column containing all relevant text data. The dataset contains 94 rows of fashion trend descriptions from authoritative fashion publications.

In [2]:
# Import required libraries
import pandas as pd
import openai

# Load the 2023 fashion trends dataset
df = pd.read_csv('2023_fashion_trends.csv')

# Display basic information about the dataset
print("Dataset shape:", df.shape)
print("\nColumn names:", df.columns.tolist())
print("\nFirst few rows:")
df.head()

Dataset shape: (82, 3)

Column names: ['URL', 'Trends', 'Source']

First few rows:


,URL,Trends,Source
0,https://www.refinery29.com/en-us/fashion-trend...,2023 Fashion Trend: Red. Glossy red hues took ...,7 Fashion Trends That Will Take Over 2023 — Sh...
1,https://www.refinery29.com/en-us/fashion-trend...,2023 Fashion Trend: Cargo Pants. Utilitarian w...,7 Fashion Trends That Will Take Over 2023 — Sh...
2,https://www.refinery29.com/en-us/fashion-trend...,"2023 Fashion Trend: Sheer Clothing. ""Bare it a...",7 Fashion Trends That Will Take Over 2023 — Sh...
3,https://www.refinery29.com/en-us/fashion-trend...,2023 Fashion Trend: Denim Reimagined. From dou...,7 Fashion Trends That Will Take Over 2023 — Sh...
4,https://www.refinery29.com/en-us/fashion-trend...,2023 Fashion Trend: Shine For The Daytime. The...,7 Fashion Trends That Will Take Over 2023 — Sh...


In [3]:
# Examine the data structure and content
print("Data types:")
print(df.dtypes)
print(f"\nTotal number of rows: {len(df)}")
print(f"Non-null values in each column:")
print(df.isnull().sum())

# Look at a few sample trend descriptions
print("\nSample trend descriptions:")
for i, trend in enumerate(df['Trends'].head(3)):
    print(f"\n{i+1}. {trend[:200]}...")

Data types:
URL       object
Trends    object
Source    object
dtype: object

Total number of rows: 82
Non-null values in each column:
URL       0
Trends    0
Source    0
dtype: int64

Sample trend descriptions:

1. 2023 Fashion Trend: Red. Glossy red hues took over the Fall 2023 runways ranging from Sandy Liang and PatBo to Tory Burch and Wiederhoeft. Think: Juicy reds with vibrant orange undertones that would l...

2. 2023 Fashion Trend: Cargo Pants. Utilitarian wear is in for 2023, which sets the stage for the return of the cargo pant. But these aren't the shapeless, low-rise pants of the Y2K era. For spring, this...

3. 2023 Fashion Trend: Sheer Clothing. "Bare it all" has been the motto since the end of the lockdown. In 2023,  naked dressing makes its way from the red carpet – where celebrities like Cher and Rihanna...


In [4]:
# Create the required 'text' column by combining relevant information
# We'll use the 'Trends' column as our primary text data since it contains the detailed fashion trend descriptions
df['text'] = df['Trends'].copy()

# Clean the text data - remove extra quotes and normalize whitespace
df['text'] = df['text'].str.replace('"', '').str.replace('""', '').str.strip()

# Verify we have at least 20 rows with text data
print(f"Number of rows with text data: {len(df)}")
print(f"Average text length: {df['text'].str.len().mean():.0f} characters")
print(f"Minimum text length: {df['text'].str.len().min()} characters")
print(f"Maximum text length: {df['text'].str.len().max()} characters")

# Display the final dataframe structure
print(f"\nFinal dataframe columns: {df.columns.tolist()}")
print(f"Sample text entries:")
for i, text in enumerate(df['text'].head(2)):
    print(f"\nEntry {i+1}: {text[:150]}...")

# Verify requirements are met
assert len(df) >= 20, "Dataset must have at least 20 rows"
assert 'text' in df.columns, "Dataset must have a 'text' column"
print(f"\n✓ Requirements met: {len(df)} rows with 'text' column created successfully!")

Number of rows with text data: 82
Average text length: 433 characters
Minimum text length: 148 characters
Maximum text length: 810 characters

Final dataframe columns: ['URL', 'Trends', 'Source', 'text']
Sample text entries:

Entry 1: 2023 Fashion Trend: Red. Glossy red hues took over the Fall 2023 runways ranging from Sandy Liang and PatBo to Tory Burch and Wiederhoeft. Think: Juic...

Entry 2: 2023 Fashion Trend: Cargo Pants. Utilitarian wear is in for 2023, which sets the stage for the return of the cargo pant. But these aren't the shapeles...

✓ Requirements met: 82 rows with 'text' column created successfully!


## Custom Query Completion

In this section, I implement the custom query system that integrates the fashion trends dataset with OpenAI's Completion model. The system creates specialized prompts that incorporate fashion trend context to provide expert-level fashion advice.

In [14]:
def basic_openai_query(prompt, max_tokens=150, temperature=0.7):
    """
    Send a basic query to OpenAI without custom data
    """
    try:
        response = client.chat.completions.create(
            model="gpt-4.1-mini",  # or "gpt-4o", "gpt-3.5-turbo"
            messages=[
                {"role": "system", "content": "You are a helpful assistant who gives clear and concise answers."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=max_tokens,
            temperature=temperature
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error: {str(e)}"


In [15]:
def create_custom_prompt(user_question, fashion_data, max_context_length=1500):
    """
    Create a custom prompt that incorporates fashion trend data
    """
    # Select relevant fashion trend information
    context_texts = []
    total_length = 0
    
    for text in fashion_data['text']:
        if total_length + len(text) < max_context_length:
            context_texts.append(text)
            total_length += len(text)
        else:
            break
    
    # Combine context
    context = "\n\n".join(context_texts)
    
    # Create the custom prompt
    custom_prompt = f"""You are a fashion expert advisor with access to the latest 2023 fashion trends and styling insights from top fashion publications. Use the following fashion trend information to provide expert advice:

FASHION TRENDS CONTEXT:
{context}

Based on this expert fashion knowledge, please answer the following question with specific, trend-aware advice:

Question: {user_question}

Answer:"""
    
    return custom_prompt

# Test the custom prompt creation
test_question = "What colors should I wear this year?"
custom_prompt = create_custom_prompt(test_question, df)
print("Custom prompt length:", len(custom_prompt))
print("Sample of custom prompt:")
print(custom_prompt[:500] + "..." if len(custom_prompt) > 500 else custom_prompt)

Custom prompt length: 1579
Sample of custom prompt:
You are a fashion expert advisor with access to the latest 2023 fashion trends and styling insights from top fashion publications. Use the following fashion trend information to provide expert advice:

FASHION TRENDS CONTEXT:
2023 Fashion Trend: Red. Glossy red hues took over the Fall 2023 runways ranging from Sandy Liang and PatBo to Tory Burch and Wiederhoeft. Think: Juicy reds with vibrant orange undertones that would look just as good in head-to-toe looks (see: a pantsuit) as accent accessor...


In [10]:
!pip install python-dotenv --quiet


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
from openai import OpenAI
import os
from dotenv import load_dotenv
# Load environment variables
load_dotenv()
client = OpenAI(
    api_key=os.getenv("VOCERUM_API_KEY"),  # Replace with your actual API key
    base_url=os.getenv("VOCERUM_BASE_URL"), 
)

def fashion_advisor_query(user_question, fashion_data, max_tokens=200, temperature=0.7):
    """
    Custom query function that provides fashion advice using the dataset
    """
    # Create custom prompt with fashion context
    custom_prompt = create_custom_prompt(user_question, fashion_data)

    try:
        response = client.chat.completions.create(
            model="gpt-4.1-mini",  # or "gpt-4o" / "gpt-4o-mini"
            messages=[
                {"role": "system", "content": "You are a fashion advisor who gives personalized outfit suggestions."},
                {"role": "user", "content": custom_prompt}
            ],
            max_tokens=max_tokens,
            temperature=temperature
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error: {str(e)}"

# Test the custom fashion advisor
test_question = "What should I wear for a spring outfit?"
print("Testing Fashion Advisor Query:")
print("Question:", test_question)
print("Custom Response:", fashion_advisor_query(test_question, df))


Testing Fashion Advisor Query:
Question: What should I wear for a spring outfit?
Custom Response: For a stylish and on-trend spring outfit in 2023, I recommend combining these key trends for a fresh, fashion-forward look:

Start with a pair of tailored cargo pants in an unexpected color—think soft pastel shades like lavender or blush, or even a sleek white or cream. Opt for cargos made from elevated fabrics such as silk or organza for a polished, sophisticated feel that moves beyond utilitarian basics.

Pair the pants with a sheer blouse or top to embrace the “naked dressing” trend. Choose a delicate organza or chiffon blouse in a neutral or soft tone, layering it over a chic bralette or camisole in a complementary color. This adds a subtle touch of sensuality while keeping the look elegant and wearable for spring.

Incorporate a pop of glossy red—a key color for 2023—through your accessories. Think vibrant red shoes, such as strappy sandals or sleek mules, or a red handbag with juicy 

In [16]:
def compare_responses(question):
    """
    Compare basic OpenAI response vs custom fashion advisor response
    """
    print("="*60)
    print(f"QUESTION: {question}")
    print("="*60)
    
    # Get basic response
    print("\n🤖 BASIC OPENAI RESPONSE:")
    print("-" * 30)
    basic_response = basic_openai_query(question)
    print(basic_response)
    
    # Get custom response
    print(f"\n✨ FASHION ADVISOR RESPONSE (with 2023 trends data):")
    print("-" * 50)
    custom_response = fashion_advisor_query(question, df)
    print(custom_response)
    
    print("\n" + "="*60)
    
    return basic_response, custom_response

# Test the comparison function
compare_responses("What are the trending colors for this year?")

QUESTION: What are the trending colors for this year?

🤖 BASIC OPENAI RESPONSE:
------------------------------
The trending colors for 2024 include:

1. **Digital Lavender** – A soft, calming purple that symbolizes wellness and digital escapism.
2. **Sundial** – A warm, earthy beige that brings a sense of grounding and balance.
3. **Verdigris** – A vibrant teal-green inspired by aged copper, reflecting a connection to nature.
4. **Tranquil Blue** – A soothing, light blue promoting calmness and clarity.
5. **Spiced Coral** – A warm, energetic coral with a hint of spice, adding vibrancy.
6. **Soft Sage** – A muted green that encourages relaxation and harmony.
7. **Warm Terracotta** – An earthy, reddish-brown tone that feels

✨ FASHION ADVISOR RESPONSE (with 2023 trends data):
--------------------------------------------------
The trending colors for 2023 prominently feature glossy, vibrant reds with orange undertones—a juicy red palette that commands attention. These reds are versatile e

('The trending colors for 2024 include:\n\n1. **Digital Lavender** – A soft, calming purple that symbolizes wellness and digital escapism.\n2. **Sundial** – A warm, earthy beige that brings a sense of grounding and balance.\n3. **Verdigris** – A vibrant teal-green inspired by aged copper, reflecting a connection to nature.\n4. **Tranquil Blue** – A soothing, light blue promoting calmness and clarity.\n5. **Spiced Coral** – A warm, energetic coral with a hint of spice, adding vibrancy.\n6. **Soft Sage** – A muted green that encourages relaxation and harmony.\n7. **Warm Terracotta** – An earthy, reddish-brown tone that feels',
 'The trending colors for 2023 prominently feature glossy, vibrant reds with orange undertones—a juicy red palette that commands attention. These reds are versatile enough to be worn head-to-toe, such as in a striking red pantsuit, or as accent pieces like shoes, handbags, and jewelry. Beyond red, the trend leans towards rich, saturated hues that complement utilita

In [17]:
# Validate the custom query system is working properly
print("CUSTOM QUERY SYSTEM VALIDATION")
print("="*40)

# Check dataset integration
print(f"✓ Dataset loaded: {len(df)} fashion trend entries")
print(f"✓ Text data available: {df['text'].notna().sum()} valid entries")
print(f"✓ Average trend description length: {df['text'].str.len().mean():.0f} characters")

# Sample trend topics covered
sample_trends = df['text'].str.extract(r'(\w+\s+\w+)(?:\s+trend|:)', expand=False).dropna().unique()[:5]
print(f"✓ Sample trend topics: {', '.join(sample_trends)}")

# Verify functions are working
try:
    test_prompt = create_custom_prompt("What should I wear?", df.head(5), max_context_length=500)
    print(f"✓ Custom prompt generation: Working (length: {len(test_prompt)} chars)")
except Exception as e:
    print(f"✗ Custom prompt generation: Error - {e}")

print(f"\n✓ Custom Fashion Advisor ready for demonstration!")
print("✓ Basic vs Custom comparison system ready!")
print("\nProceed to performance demonstration below...")

CUSTOM QUERY SYSTEM VALIDATION
✓ Dataset loaded: 82 fashion trend entries
✓ Text data available: 82 valid entries
✓ Average trend description length: 433 characters
✓ Sample trend topics: Fashion Trend, anticipated fashion, this spring, great news, we know
✓ Custom prompt generation: Working (length: 689 chars)

✓ Custom Fashion Advisor ready for demonstration!
✓ Basic vs Custom comparison system ready!

Proceed to performance demonstration below...


## Custom Performance Demonstration

This section demonstrates the clear performance improvement of the custom fashion advisor compared to basic OpenAI responses. Each question showcases how the integrated fashion dataset provides specific, trend-aware advice that significantly enhances the chatbot's value.

### Question 1

In [18]:
# Question 1: Specific trend inquiry about denim styling
question1 = "How should I style denim pieces for a modern look?"

basic_resp1, custom_resp1 = compare_responses(question1)

QUESTION: How should I style denim pieces for a modern look?

🤖 BASIC OPENAI RESPONSE:
------------------------------
To style denim pieces for a modern look, consider these tips:

1. **Mix Denim with Different Textures:** Pair denim jeans or jackets with soft knits, leather, or silk to create contrast and add interest.
2. **Opt for Tailored Fits:** Choose well-fitted denim like straight-leg or slim-fit jeans rather than overly baggy or overly tight styles for a sleek silhouette.
3. **Double Denim Done Right:** If you want to wear denim on denim, vary the shades (light wash jeans with a dark denim jacket) and add a neutral or colorful top underneath to break up the look.
4. **Add Modern Accessories:** Incorporate trendy accessories like chunky sneakers, statement belts, or minimalist jewelry to elevate your denim outfit.

✨ FASHION ADVISOR RESPONSE (with 2023 trends data):
--------------------------------------------------
To style denim pieces for a modern, on-trend 2023 look, conside

### Question 2

In [20]:
# Question 2: Color trend inquiry
question2 = "What are the must-have colors to wear this season?"

basic_resp2, custom_resp2 = compare_responses(question2)

QUESTION: What are the must-have colors to wear this season?

🤖 BASIC OPENAI RESPONSE:
------------------------------
This season's must-have colors typically include:

1. **Warm Earth Tones:** Shades like terracotta, burnt orange, and rich browns are very popular.
2. **Soft Pastels:** Light pinks, lilacs, and baby blues offer a fresh, gentle look.
3. **Vibrant Reds:** Bold reds and crimson hues make a strong fashion statement.
4. **Classic Neutrals:** Beige, cream, and soft greys remain versatile and stylish.
5. **Deep Greens:** Emerald and olive greens add a touch of sophistication.

These colors can be mixed and matched to create trendy and seasonally appropriate outfits!

✨ FASHION ADVISOR RESPONSE (with 2023 trends data):
--------------------------------------------------
The must-have colors to wear this season are vibrant, glossy reds with warm orange undertones—think juicy, attention-grabbing hues that work beautifully from head-to-toe or as striking accent pieces like shoes an

In [22]:
# Interactive fashion advisor loop
print("Welcome to the 2023 Fashion Advisor! Type your fashion question below (or type 'exit' to quit):")

while True:
    user_question = input("\nYour question: ")
    if user_question.strip().lower() in ['exit', 'quit']:
        print("Thank you for using the Fashion Advisor. Goodbye!")
        break

    # Get custom fashion advice using the dataset
    response = fashion_advisor_query(user_question, df)
    print("\nFashion Advisor Response:")
    print(response)

Welcome to the 2023 Fashion Advisor! Type your fashion question below (or type 'exit' to quit):

Fashion Advisor Response:
It looks like your question didn't come through—could you please share what specific outfit advice or styling tips you're looking for? For example, are you interested in incorporating red hues, cargo pants, or sheer clothing into your wardrobe? Let me know, and I'll provide personalized, trend-savvy recommendations based on the latest 2023 fashion insights!

Fashion Advisor Response:
For your Goa trip, embracing light, breathable, and stylish pieces that align with 2023 trends will keep you both comfortable and fashion-forward.

Here’s a trend-savvy outfit suggestion:

- **Sheer Clothing:** Opt for a lightweight sheer dress or a sheer blouse paired with a vibrant bikini or bralette underneath. This plays right into the 2023 sheer trend while keeping you cool and beach-ready. Choose airy fabrics like organza or chiffon in light or bright colors to channel that breez